In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false
}

<IPython.core.display.Javascript object>

In [3]:
# Todo: put this into local file or local db
person_dict = {'Todd':'Todd','Alice': 'Alice', 'Mitchell': 'Mitchell'}


In [4]:
from CoreConfig.CoreConfig import CoreConfig
from DBUtility.DBUtility import DBSQLiteConnection, CreateEventsTableIfNotExist
from DBUtility.DBUtility import FetchEventsRecordWithinSamePerson, FetchEventsLatestRecordWithSamePerson, FetchEventsRecordAll
from DBUtility.DBUtility import InsertEventRecord
from pandas import DataFrame
import datetime
CoreConfig.initialize_core_config()

DBSQLiteConnection().configure_connection_information(**CoreConfig().get_sqlite3_db_connection_info())
CreateEventsTableIfNotExist().execute()


In [5]:
CoreConfig.initialize_core_config()

DBSQLiteConnection().configure_connection_information(**CoreConfig().get_sqlite3_db_connection_info())
CreateEventsTableIfNotExist().execute()

Initialize nab ventures events board             table failed. Please Check your connection
table nab_ventures_events_board already exists


In [6]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, interactive_output
from IPython.display import display_pretty
import pandas as pd
from ipywidgets import Layout, Button, Box

items_layout = Layout(display='flex',
                    flex_flow='auto',
                     align_items='stretch',
                      width='auto')     # override the default width of the button to 'auto' to let the button grow



box_layout = Layout(display='flex',
                    flex_flow='auto',
                    align_items='center',
                    width='auto')

textarea_layout = Layout(display='flex',
                    flex_flow='auto',
                    align_items='center',
                    width='auto',
                    height='120px')

style = {'description_width': 'initial'}

In [7]:
title = widgets.HTML("<center><H3>Recent records</H3><center>")
formtitle = widgets.HTML("<center><H3>Submit Form</H3><center>")
df_out = widgets.Output(layout={'border': '1px solid black', 'aligh_items' : 'center'})
with df_out:
    df = DataFrame(FetchEventsLatestRecordWithSamePerson().execute(), columns=['Events Name', 'Person', 'Date'])
    display(df)
events = widgets.Text(
      value='',
      placeholder='Type Events',
      description='Events:',
      layout=box_layout,
       style=style,
      disable=False)

meeting = widgets.Checkbox(
    value=False,
    description='Board Meeting',
    disabled=False
)

date = widgets.DatePicker(
        value = datetime.datetime.now().date(),
        description='Pick a Date',
        layout=box_layout,
        style = style,
        disable=False)

name=widgets.Text(
      value='',
      placeholder='Type speaker\'s name',
      description='Speaker:',
      layout=box_layout,
       style=style,
      disable=False)
#display_pretty(name)
notes=widgets.Text(
      value='',
      placeholder='Type note/organizer',
      description='Note/ Organizer:',
      layout=box_layout,
       style=style,
      disable=False)

country = widgets.Dropdown(options= {'Australia': 'Australia',
                                     'New Zealand': 'New Zealand',
                                    'None':'None'},
                           value = 'Australia',
                           description='Select country:',
                           layout=box_layout,
                           style=style)
audience= widgets.Text(value='0',
                       placeholder='Type audience numbers',
                       description='Audience:',
                       layout=box_layout,
                       style=style,
                       disable=False)


website= widgets.Text(value='https://',
                       placeholder='Type website/context',
                       description='Website:',
                       layout=box_layout,
                       style=style,
                       disable=False)
asset = widgets.Text(value='',
                       placeholder='Type asset',
                       description='Asset:',
                       layout=box_layout,
                       style=style,
                       disable=False)

person = widgets.Dropdown(
                options= person_dict,
                value='Todd',
                description='Select your person:',
                layout=box_layout,
                style=style
              )
new_person_check = widgets.Checkbox(
    value=False,
    description='Add new person',
    disable = False
)
new_person_text = widgets.Text(
      value='',
      placeholder='Type your name',
      description='New person to be added:',
      layout=box_layout,
       style=style,
      disabled=True)

new_person_button = widgets.Button(
    description='Submit',
    disabled=True,
    button_style ='', # success
    tooltip='Click me',
    icon='check')



new_person_box = widgets.HBox([new_person_check, new_person_text, new_person_button])

#business_problem = widgets.Textarea(
#                   value='',
#                    placeholder='Type your business problem here',
#                    description='Describe your use case or what is your business problem?:',
#                    layout=textarea_layout,
#                    style=style,
#                    disable=False)


button = widgets.Button(
           description='Submit the form',
           disable=False,
           button_style ='', # success
           tooltip='Click me',
           icon='check'
           )

def on_add_new_name_value_change(change):
    if change['new'] == True:
        new_person_text.disabled=False
        new_person_button.disabled=False
    else:
        new_person_text.disabled=True
        new_person_button.disabled=True

def on_submit_add_new_name_button_clicked(b):
    '''Check the local person_dict, if not included, 
    then added and refresh
    '''
    new_person = new_person_text.value
    person.options[new_person] = new_person
    new_person_check.value = False
    new_person_text.disabled=True
    new_person_button.disabled=True
        
new_person_check.observe(on_add_new_name_value_change, names='value')
new_person_button.on_click(on_submit_add_new_name_button_clicked)
def on_submit_form_button_clicked(b):
    outputs = {'events': [events.value],
                'meeting' : [meeting.value],
                'date': [date.value],
                'name': [name.value],
                'notes': [notes.value],
                'country': [country.value],
                'audience': [audience.value], 
                'website': [website.value],
                'asset' : [asset.value],
                'person' : [person.value]
               }
    InsertEventRecord(events.value, meeting.value, date.value, name.value, notes.value,
                      country.value, audience.value, website.value, asset.value, person.value).execute()
    df = DataFrame(FetchEventsLatestRecordWithSamePerson().execute(), columns=['Events Name', 'Person', 'Date'])
    df_out.clear_output()
    with df_out:
        display(df)
button.on_click(on_submit_form_button_clicked)

In [10]:
def form(events, meeting, date, name, notes, country, audience, website, asset, person, new_person_name):
    #display(events, meeting, date, name, notes, country, audience, website, asset, person)
    return events, meeting, date, name, notes, country, audience, website, asset, person, new_person_name

In [8]:
ui = widgets.VBox(children=[title,
                            df_out,
                            formtitle,
                            events,
                            meeting,
                            date,
                            name,
                            notes,
                            country,
                            audience,
                            website,
                            asset,
                            person,
                            button,
                            new_person_box], layout=items_layout)

In [12]:
out = interactive_output(form, {'events': events,
                                'meeting' : meeting,
                                'date':date,
                                'name':name,
                                'notes':notes,
                                'country':country,
                                'audience': audience, 
                                'website': website,
                                'asset' : asset,
                                'person' : person,
                                'new_person_name' : new_person_box.children[1]
                               })
display(ui, out)

Output()

In [ ]:
out.outputs

In [ ]:
tables = FetchEventsRecordWithinSamePerson('Alice').execute()
for item in tables:
    display(item)

In [ ]:
test_value

In [ ]:
def g(x):
    display(x)
    return x

h = interactive(g, x=10);

In [ ]:
out.chidren[0]

In [ ]:
h.result

In [ ]:
out1 = widgets.Output(layout={'border': '1px solid black'})

In [ ]:
out1

In [ ]:
with out1:
    for i in range(10):
        print(i, 'Hello world!')

In [ ]:
out1.outputs

In [ ]:
import pandas as pd

df = pd.read_csv('test.csv')
import qgrid
qgrid_widget = qgrid.show_grid(df, show_toolbar=True)
qgrid_widget

### Fake

### Experiment

In [ ]:
a = widgets.IntSlider(description='a', disabled=True)
b = widgets.IntSlider(description='b')
c = widgets.IntSlider(description='c')
def f(a, b, c):
    print('{}*{}*{}={}'.format(a, b, c, a*b*c))

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

widgets.HBox([widgets.VBox([a, b, c]), out])

In [ ]:
# output widget
df = DataFrame(FetchEventsLatestRecordWithSamePerson().execute())

with df_out:
    display(df)

In [ ]:
check_exper = widgets.Checkbox(
    value=False,
    description='Add new person',
    disable = False)
person_text_exper = widgets.Text(
      value='',
      placeholder='Type your name',
      description='New person to be added:',
      layout=box_layout,
       style=style,
      disabled=True)
display(check_exper, person_text_exper)

def on_value_change(change):
    if change['new'] == True:
        person_text_exper.disabled=False
    else:
        person_text_exper.disabled=True

check_exper.observe(on_value_change, names='value')

In [ ]:
expr_country = widgets.Dropdown(options= {'Australia': 'Australia',
                                     'New Zealand': 'New Zealand',
                                    'None':'None'},
                                   value = 'Australia',
                               description='Select country:',
                               layout=box_layout,
                               style=style)
display(expr_country)

In [ ]:
expr_country.options={'Australia': 'Australia',
                                     'New Zealand': 'New Zealand'}

In [ ]:
expr_country.options={'Australia': 'Australia',
                                     'New Zealand': 'New Zealand', 'Taiwan' : 'Taiwan'}